In [1]:
line="{{product_name:浙江在线杭州}}{{time:4月25日}}讯（记者{{person_name: 施宇翔}} 通讯员 {{person_name:方英}}）毒贩很“时髦”，用{{product_name:微信}}交易毒品。没料想警方也很“潮”，将计就计，一举将其擒获。记者从{{org_name:杭州江干区公安分局}}了解到，经过一个多月的侦查工作，{{org_name:江干区禁毒专案组}}抓获吸贩毒人员5名，缴获“冰毒”400余克，毒资30000余元，扣押汽车一辆。{{location:黑龙江}}籍男子{{person_name:钱某}}长期落脚于宾馆、单身公寓，经常变换住址。他有一辆车，经常半夜驾车来往于{{location:杭州主城区}}的各大宾馆和单身公寓，并且常要活动到{{time:凌晨6、7点钟}}，{{time:白天}}则在家里呼呼大睡。{{person_name:钱某}}不寻常的特征，引起了警方注意。禁毒大队通过侦查，发现{{person_name:钱某}}实际上是在向落脚于宾馆和单身公寓的吸毒人员贩送“冰毒”。"
entity=["time","location","person_name","org_name","company_name","product_name"]
import re,jieba


In [4]:
newline=[]
loc_entity=[]
person_entity=[]
org_entity=[]
doubelsurname=['欧阳', '太史', '端木', '上官', '司马', '东方', '独孤', '南宫', '万俟', '闻人', '夏侯', '诸葛', '尉迟', '公羊', '赫连', '澹台', '皇甫', '宗政', '濮阳', '公冶', '太叔', '申屠', '公孙', '慕容', '仲孙', '钟离', '长孙', '宇文', '司徒', '鲜于', '司空', '闾丘', '子车', '亓官', '司寇', '巫马', '公西', '颛孙', '壤驷', '公良', '漆雕', '乐正', '宰父', '谷梁', '拓跋', '夹谷', '轩辕', '令狐', '段干', '百里', '呼延', '东郭', '南门', '羊舌', '微生', '公户', '公玉', '公仪', '梁丘', '公仲', '公上', '公门', '公山', '公坚', '左丘', '公伯', '西门', '公祖', '第五', '公乘', '贯丘', '公皙', '南荣', '东里', '东宫', '仲长', '子书', '子桑', '即墨', '达奚', '褚师', '吴铭']

def append_entity(entity,newline):
    entity=list(jieba.cut(entity))
    for ent in entity:
        newline.append(ent)
    return newline,entity

def perEntity(entity):
    name=[]
    length=len(entity)
    if length!=1:
        if entity[0] not in doubelsurname and len(entity[0])==2:
            name.append("C")
            
context_u=False #上下文的上文
context_d=False #上下文的下文
compile=re.compile("{{(.*?)}}")
filter_compile=re.compile("\w+:")

split_line=compile.split(line)
for ind,lin in enumerate(split_line):
    if len(lin)==0:
        continue
    if filter_compile.match(lin):
        entity_name,entity=lin.split(":")
        newline,entity_cut=append_entity(entity,newline)
        entity_len=len(entity_cut)
        if entity_name=='person_name':
            if entity_len==1:
                if entity[:2] in doubelsurname:
                    person_entity.append("C")
                    if len(entity[2:])==1:
                        person_entity.append("F")
                    else:
                        for _ in range(len(entity[2:-1])):
                            person_entity.append("D")
                        person_entity.append("E")
                else:                   
                    person_entity.append("C")
                    if len(entity[1:])>=2:
                        for _ in range(len(entity[1:-1])):
                            person_entity.append("D")
                        person_entity.append("E")
                    else:
                        person_entity.append("F")
            else:
                
        context_d=True
    else:
        newline,_=append_entity(lin,newline)
        newline_len=len(newline)
        # 判断是否是连接词
        if context_d and filter_compile.match(split_line[ind+1]) and newline_len==1:
            person_entity.append("X")
            continue
        # 其他非连接词
        if context_d and newline_len>1:
            person_entity.append("B")
            for _ in range(len(newline[1:-1])):
                person_entity.append("Z")
            if filter_compile.match(split_line[ind+1]):
                person_entity.append("A")
            else:
                person_entity.append("Z")
            

liness=' '.join(compile.split(line))
print(list(jieba.cut(liness)))





黑龙江
杭州
主城区
[' ', 'product', '_', 'name', ':', '浙江', '在线', '杭州', ' ', ' ', 'time', ':', '4', '月', '25', '日', ' ', '讯', '（', '记者', ' ', 'person', '_', 'name', ':', ' ', '施宇翔', ' ', ' ', '通讯员', ' ', ' ', 'person', '_', 'name', ':', '方英', ' ', '）', '毒贩', '很', '“', '时髦', '”', '，', '用', ' ', 'product', '_', 'name', ':', '微信', ' ', '交易', '毒品', '。', '没', '料想', '警方', '也', '很', '“', '潮', '”', '，', '将计就计', '，', '一举', '将', '其', '擒获', '。', '记者', '从', ' ', 'org', '_', 'name', ':', '杭州', '江干区', '公安分局', ' ', '了解', '到', '，', '经过', '一个多月', '的', '侦查', '工作', '，', ' ', 'org', '_', 'name', ':', '江干区', '禁毒', '专案组', ' ', '抓获', '吸', '贩毒', '人员', '5', '名', '，', '缴获', '“', '冰毒', '”', '400', '余克', '，', '毒资', '30000', '余元', '，', '扣押', '汽车', '一辆', '。', ' ', 'location', ':', '黑龙江', ' ', '籍', '男子', ' ', 'person', '_', 'name', ':', '钱', '某', ' ', '长期', '落脚', '于', '宾馆', '、', '单身公寓', '，', '经常', '变换', '住址', '。', '他', '有', '一辆车', '，', '经常', '半夜', '驾车', '来往', '于', ' ', 'location', ':', '杭州', '主城区', ' ', '的', '各大', '宾馆', '和'

['张', '东北', '男', '黑红', '姐']

In [6]:
print(list(jieba.cut("纯种波尔山羊")))

['纯种', '波尔', '山羊']


In [18]:
files="/Users/apple/Downloads/现代汉语词典（第五版）全文.txt"
f=open(files,'r',encoding='utf-16')
lines=f.readlines()
compiles=re.compile("【(.*?)】")
for line in lines:
    # line=line.decode("gbk").encode('utf-8')
    print(compiles.findall(line))

[]
[]
[]
[]
[]
[]
[]
[]
['吖']
[]
['阿']
[]
[]
['阿鼻地狱']
[]
['阿昌族']
[]
['阿斗']
[]
['阿尔茨海默病']
[]
['阿尔法粒子']
[]
['阿尔法射线']
[]
['阿飞']
[]
['阿伏伽德罗常量']
[]
['阿公']
[]
['阿訇']
[]
['阿拉伯人']
[]
['阿拉伯数字']
[]
['阿兰若']
[]
['阿罗汉']
[]
['阿猫阿狗']
[]
['阿门']
[]
['阿片']
[]
['阿婆']
[]
['阿Q']
[]
['阿是穴']
[]
['阿嚏']
[]
['阿姨']
[]
['啊']
[]
[]
['锕']
[]
['腌']
[]
[]
['啊']
[]
[]
['嗄']
[]
[]
['啊']
[]
[]
['啊']
[]
[]
['啊']
[]
[]
['哎']
[]
['哎呀']
[]
['哎哟']
[]
['哀']
[]
['哀兵必胜']
[]
['哀愁']
[]
['哀辞']
[]
['哀悼']
[]
['哀的美敦书']
[]
['哀告']
[]
['哀歌']
[]
['哀号']
[]
['哀嚎']
[]
['哀鸿遍野']
[]
['哀矜']
[]
['哀苦']
[]
['哀怜']
[]
['哀鸣']
[]
['哀戚']
[]
['哀启']
[]
['哀泣']
[]
['哀切']
[]
['哀求']
[]
['哀荣']
[]
['哀伤']
[]
['哀思']
[]
['哀叹']
[]
['哀恸']
[]
['哀痛']
[]
['哀婉']
[]
['哀艳']
[]
['哀怨']
[]
['哀乐']
[]
['埃']
[]
['埃']
[]
['埃博拉出血热']
[]
['挨']
[]
[]
['挨边']
[]
['挨次']
[]
['挨个儿']
[]
['挨肩儿']
[]
['挨近']
[]
['唉']
[]
[]
['唉声叹气']
[]
['娭']
[]
[]
['欸']
[]
[]
['嗳']
[]
[]
['锿']
[]
['挨']
[]
[]
['挨板子']
[]
['挨批']
[]
['挨宰']
[]
['挨整']
[]
['𫘤']
[]
['皑']
[]
['皑皑']
[]
['癌']
[]
['癌变']
[]
['癌症']
[]
['毐'

[]
['白契']
[]
['白镪']
[]
['白区']
[]
['白饶']
[]
['白热']
[]
['白热化']
[]
['白人']
[]
['白刃']
[]
['白刃战']
[]
['白日']
[]
['白日见鬼']
[]
['白日做梦']
[]
['白肉']
[]
['白润']
[]
['白色']
[]
['白色恐怖']
[]
['白色垃圾']
[]
['白色收入']
[]
['白色污染']
[]
['白山黑水']
[]
['白鳝']
[]
['白食']
[]
['白事']
[]
['白手']
[]
['白手起家']
[]
['白首']
[]
['白薯']
[]
['白水']
[]
['白苏']
[]
['白汤']
[]
['白糖']
[]
['白陶']
[]
['白体']
[]
['白天']
[]
['白田']
[]
['白条']
[]
['白条']
[]
['白铁']
[]
['白厅']
[]
['白铜']
[]
['白头']
[]
['白头']
[]
['白头鹎']
[]
['白头翁']
[]
['白头偕老']
[]
['白玩儿']
[]
['白文']
[]
['白皙']
[]
['白细胞']
[]
['白鹇']
[]
['白鲞']
[]
['白相']
[]
['白熊']
[]
['白血病']
[]
['白血球']
[]
['白眼']
[]
['白眼儿狼']
[]
['白眼珠']
[]
['白羊座']
[]
['白杨']
[]
['白药']
[]
['白页']
[]
['白夜']
[]
['白衣苍狗']
[]
['白衣天使']
[]
['白衣战士']
[]
['白蚁']
[]
['白翳']
[]
['白银']
[]
['白玉']
[]
['白云苍狗']
[]
['白云岩']
[]
['白灾']
[]
['白斩鸡']
[]
['白芷']
[]
['白纸黑字']
[]
['白质']
[]
['白雉']
[]
['白种']
[]
['白昼']
[]
['白术']
[]
['白字']
[]
['白族']
[]
['白嘴儿']
[]
['百']
[]
['百般']
[]
['百宝箱']
[]
['百倍']
[]
['百步穿杨']
[]
['百尺竿头，更进一步']
[]
['百出']
[]
['百川归海']
[]
['百儿八十']
[]
['百发百中']
[]

['帮补']
[]
['帮衬']
[]
['帮厨']
[]
['帮凑']
[]
['帮带']
[]
['帮倒忙']
[]
['帮扶']
[]
['帮工']
[]
['帮会']
[]
['帮教']
[]
['帮困']
[]
['帮忙']
[]
['帮派']
[]
['帮腔']
[]
['帮手']
[]
['帮套']
[]
['帮贴']
[]
['帮闲']
[]
['帮凶']
[]
['帮佣']
[]
['帮主']
[]
['帮助']
[]
['帮子']
[]
['帮子']
[]
['𠳐']
[]
['𠳐啷']
[]
['梆']
[]
['梆硬']
[]
['梆子']
[]
['梆子腔']
[]
['浜']
[]
['绑']
[]
['绑匪']
[]
['绑缚']
[]
['绑架']
[]
['绑票']
[]
['绑腿']
[]
['绑扎']
[]
['榜']
[]
[]
['榜额']
[]
['榜首']
[]
['榜书']
[]
['榜尾']
[]
['榜文']
[]
['榜眼']
[]
['榜样']
[]
['牓']
[]
['膀']
[]
[]
['膀臂']
[]
['膀大腰圆']
[]
['膀子']
[]
['髈']
[]
[]
['蚌']
[]
[]
['棒']
[]
['棒冰']
[]
['棒疮']
[]
['棒槌']
[]
['棒喝']
[]
['棒球']
[]
['棒儿香']
[]
['棒针']
[]
['棒子']
[]
['棒子面']
[]
['棓']
[]
[]
['傍']
[]
['傍边儿']
[]
['傍黑儿']
[]
['傍角儿']
[]
['傍亮儿']
[]
['傍明']
[]
['傍人门户']
[]
['傍晌']
[]
['傍晚']
[]
['傍午']
[]
['傍依']
[]
['谤']
[]
['谤书']
[]
['谤议']
[]
['塝']
[]
['搒']
[]
[]
['蒡']
[]
[]
['稖']
[]
['蜯']
[]
['膀']
[]
[]
['磅']
[]
[]
['磅秤']
[]
['镑']
[]
['艕']
[]
['包']
[]
['包办']
[]
['包保']
[]
['包庇']
[]
['包藏']
[]
['包藏祸心']
[]
['包产']
[]
['包场']
[]
['包抄']
[]
['包车']
[]
['


[]
['爆发音']
[]
['爆冷']
[]
['爆冷门']
[]
['爆料']
[]
['爆裂']
[]
['爆满']
[]
['爆棚']
[]
['爆破']
[]
['爆破筒']
[]
['爆胎']
[]
['爆笑']
[]
['爆炸']
[]
['爆炸性']
[]
['爆仗']
[]
['爆竹']
[]
['陂']
[]
[]
['杯']
[]
['杯葛']
[]
['杯弓蛇影']
[]
['杯珓']
[]
['杯盘狼藉']
[]
['杯赛']
[]
['杯水车薪']
[]
['杯中物']
[]
['杯子']
[]
['卑']
[]
['卑鄙']
[]
['卑不足道']
[]
['卑词']
[]
['卑辞']
[]
['卑躬屈节']
[]
['卑躬屈膝']
[]
['卑贱']
[]
['卑劣']
[]
['卑怯']
[]
['卑俗']
[]
['卑微']
[]
['卑污']
[]
['卑下']
[]
['卑职']
[]
['背']
[]
[]
['背榜']
[]
['背包']
[]
['背包袱']
[]
['背带']
[]
['背篼']
[]
['背负']
[]
['背黑锅']
[]
['背饥荒']
[]
['背筐']
[]
['背篓']
[]
['背头']
[]
['背债']
[]
['背子']
[]
['椑']
[]
[]
['悲']
[]
['悲哀']
[]
['悲惨']
[]
['悲愁']
[]
['悲楚']
[]
['悲怆']
[]
['悲悼']
[]
['悲愤']
[]
['悲歌']
[]
['悲观']
[]
['悲号']
[]
['悲欢离合']
[]
['悲剧']
[]
['悲苦']
[]
['悲凉']
[]
['悲悯']
[]
['悲鸣']
[]
['悲凄']
[]
['悲戚']
[]
['悲泣']
[]
['悲切']
[]
['悲情']
[]
['悲伤']
[]
['悲声']
[]
['悲酸']
[]
['悲叹']
[]
['悲天悯人']
[]
['悲恸']
[]
['悲痛']
[]
['悲喜交集']
[]
['悲喜剧']
[]
['悲辛']
[]
['悲咽']
[]
['悲壮']
[]
['碑']
[]
['碑额']
[]
['碑记']
[]
['碑碣']
[]
['碑刻']
[]
['碑林']
[]
['碑铭']
[]
['碑首']


[]
['\ue6a4']
[]
['陛']
[]
['陛下']
[]
['𫖒']
[]
['毙']
[]
['毙命']
[]
['毙伤']
[]
['铋']
[]
['秘']
[]
[]
['狴']
[]
['萆']
[]
['萆薢']
[]
['梐']
[]
['庳']
[]
['敝']
[]
['敝人']
[]
['敝屣']
[]
['敝帚千金']
[]
['敝帚自珍']
[]
['婢']
[]
['婢女']
[]
['皕']
[]
['赑']
[]
['筚']
[]
['筚篥']
[]
['筚路蓝缕']
[]
['湢']
[]
['愊']
[]
['愎']
[]
['弼']
[]
['蓖']
[]
['跸']
[]
['腷']
[]
['痹']
[]
['痹症']
[]
['煏']
[]
['滗']
[]
['裨']
[]
[]
['裨益']
[]
['辟']
[]
['辟']
[]
['辟']
[]
[]
['辟谷']
[]
['辟邪']
[]
['辟易']
[]
['碧']
[]
['碧波']
[]
['碧空']
[]
['碧蓝']
[]
['碧绿']
[]
['碧螺春']
[]
['碧落']
[]
['碧血']
[]
['碧油油']
[]
['碧玉']
[]
['蔽']
[]
['蔽芾']
[]
['蔽塞']
[]
['蔽障']
[]
['馝']
[]
['箅']
[]
['弊']
[]
['弊病']
[]
['弊端']
[]
['弊害']
[]
['弊绝风清']
[]
['弊政']
[]
['髲']
[]
['獘']
[]
['薜']
[]
['觱']
[]
['篦']
[]
['篦子']
[]
['壁']
[]
['壁报']
[]
['壁布']
[]
['壁橱']
[]
['壁灯']
[]
['壁挂']
[]
['壁柜']
[]
['壁虎']
[]
['壁画']
[]
['壁垒']
[]
['壁垒森严']
[]
['壁立']
[]
['壁炉']
[]
['壁球']
[]
['壁上观']
[]
['壁虱']
[]
['壁式网球']
[]
['壁饰']
[]
['壁毯']
[]
['壁厢']
[]
['壁障']
[]
['壁纸']
[]
['壁钟']
[]
['避']
[]
['避风']
[]
['避风港']
[]
['避风头']
[]
['避讳']


...

['屌']
[]
['吊']
[]
['吊']
[]
['吊']
[]
['吊膀子']
[]
['吊车']
[]
['吊窗']
[]
['吊床']
[]
['吊带']
[]
['吊灯']
[]
['吊顶']
[]
['吊儿郎当']
[]
['吊环']
[]
['吊祭']
[]
['吊脚楼']
[]
['吊卷']
[]
['吊扣']
[]
['吊兰']
[]
['吊楼']
[]
['吊毛']
[]
['吊民伐罪']
[]
['吊铺']
[]
['吊钱儿']
[]
['吊桥']
[]
['吊丧']
[]
['吊嗓子']
[]
['吊扇']
[]
['吊桶']
[]
['吊袜带']
[]
['吊胃口']
[]
['吊线']
[]
['吊销']
[]
['吊孝']
[]
['吊唁']
[]
['吊装']
[]
['吊子']
[]
['钓']
[]
['钓饵']
[]
['钓竿']
[]
['钓钩']
[]
['钓具']
[]
['𦰏']
[]
['窎']
[]
['窎远']
[]
['调']
[]
['调']
[]
[]
['调包']
[]
['调兵遣将']
[]
['调拨']
[]
[]
['调茬']
[]
['调查']
[]
['调档']
[]
['调调']
[]
['调动']
[]
['调度']
[]
['调防']
[]
['调干']
[]
['调函']
[]
['调号']
[]
['调虎离山']
[]
['调换']
[]
['调集']
[]
['调卷']
[]
['调坎儿']
[]
['调侃儿']
[]
['调类']
[]
['调离']
[]
['调令']
[]
['调门儿']
[]
['调派']
[]
['调配']
[]
[]
['调遣']
[]
['调任']
[]
['调式']
[]
['调头']
[]
['调头']
[]
['调研']
[]
['调演']
[]
['调用']
[]
['调阅']
[]
['调运']
[]
['调值']
[]
['调职']
[]
['调转']
[]
['调子']
[]
['掉']
[]
['掉']
[]
['掉包']
[]
['掉膘']
[]
['掉秤']
[]
['掉点儿']
[]
['掉队']
[]
['掉过儿']
[]
['掉换']
[]
['掉价']
[]
['掉枪花']
[]
['掉色']
[]
['掉书袋']
[]
[


[]
['短缺']
[]
['短日照植物']
[]
['短少']
[]
['短视']
[]
['短途']
[]
['短线']
[]
['短小']
[]
['短小精悍']
[]
['短信']
[]
['短信息']
[]
['短语']
[]
['短暂']
[]
['短装']
[]
['段']
[]
['段落']
[]
['段位']
[]
['段子']
[]
['断']
[]
['断']
[]
['断案']
[]
['断案']
[]
['断壁残垣']
[]
['断编残简']
[]
['断层']
[]
['断肠']
[]
['断炊']
[]
['断代']
[]
['断代']
[]
['断代史']
[]
['断档']
[]
['断定']
[]
['断断']
[]
['断断续续']
[]
['断顿']
[]
['断根']
[]
['断喝']
[]
['断后']
[]
['断后']
[]
['断乎']
[]
['断魂']
[]
['断简残编']
[]
['断简残篇']
[]
['断交']
[]
['断井颓垣']
[]
['断句']
[]
['断绝']
[]
['断粮']
[]
['断裂']
[]
['断流']
[]
['断垄']
[]
['断路']
[]
['断码']
[]
['断面']
[]
['断奶']
[]
['断片']
[]
['断七']
[]
['断气']
[]
['断然']
[]
['断市']
[]
['断送']
[]
['断头台']
[]
['断弦']
[]
['断线风筝']
[]
['断想']
[]
['断言']
[]
['断语']
[]
['断狱']
[]
['断垣残壁']
[]
['断章取义']
[]
['断种']
[]
['断子绝孙']
[]
['塅']
[]
['缎']
[]
['缎子']
[]
['椴']
[]
['煅']
[]
['煅烧']
[]
['碫']
[]
['锻']
[]
['锻锤']
[]
['锻工']
[]
['锻件']
[]
['锻炼']
[]
['锻铁']
[]
['锻压']
[]
['锻造']
[]
['簖']
[]
['堆']
[]
['堆叠']
[]
['堆房']
[]
['堆放']
[]
['堆肥']
[]
['堆积']
[]
['堆集']
[]
['堆砌']
[]
['堆笑']
[]
['堆栈']
[]
['𫗰']
[]


[]
['二郎腿']
[]
['二老']
[]
['二愣子']
[]
['二流']
[]
['二流子']
[]
['二毛']
[]
['二门']
[]
['二面角']
[]
['二拇指']
[]
['二奶']
[]
['二年生']
[]
['二七大罢工']
[]
['二人台']
[]
['二人转']
[]
['二审']
[]
['二十八宿']
[]
['二十四节气']
[]
['二十四史']
[]
['二十五史']
[]
['二手']
[]
['二踢脚']
[]
['二五眼']
[]
['二弦']
[]
['二线']
[]
['二心']
[]
['二氧化氮']
[]
['二氧化硅']
[]
['二氧化硫']
[]
['二氧化碳']
[]
['二一添作五']
[]
['二意']
[]
['二元论']
[]
['二战']
[]
['弍']
[]
['刵']
[]
['佴']
[]
[]
['贰']
[]
['贰臣']
[]
['贰心']
[]
['咡']
[]
['樲']
[]
[]
['发']
[]
[]
['发案']
[]
['发榜']
[]
['发包']
[]
['发报']
[]
['发标']
[]
['发表']
[]
['发兵']
[]
['发病']
[]
['发布']
[]
['发财']
[]
['发车']
[]
['发痴']
[]
['发愁']
[]
['发出']
[]
['发憷']
[]
['发达']
[]
['发达国家']
[]
['发呆']
[]
['发嗲']
[]
['发电']
[]
['发电机']
[]
['发动']
[]
['发动机']
[]
['发抖']
[]
['发端']
[]
['发凡']
[]
['发放']
[]
['发粉']
[]
['发奋']
[]
['发愤']
[]
['发愤图强']
[]
['发疯']
[]
['发福']
[]
['发付']
[]
['发绀']
[]
['发糕']
[]
['发稿']
[]
['发光强度']
[]
['发汗']
[]
['发行']
[]
[]
['发号施令']
[]
['发狠']
[]
['发横']
[]
['发花']
[]
['发话']
[]
['发话器']
[]
['发还']
[]
['发慌']
[]
['发挥']
[]
['发昏']
[]
['发火']
[]
['发急']
[]
['发迹']



[]
[]
['坊本']
[]
['坊间']
[]
['芳']
[]
['芳菲']
[]
['芳邻']
[]
['芳龄']
[]
['芳名']
[]
['芳年']
[]
['芳容']
[]
['芳烃']
[]
['芳香']
[]
['芳心']
[]
['芳馨']
[]
['芳泽']
[]
['枋']
[]
['枋']
[]
['枋子']
[]
['钫']
[]
['钫']
[]
['蚄']
[]
['防']
[]
['防暴']
[]
['防备']
[]
['防不胜防']
[]
['防潮']
[]
['防尘']
[]
['防除']
[]
['防弹']
[]
['防盗']
[]
['防盗门']
[]
['防地']
[]
['防冻']
[]
['防毒']
[]
['防毒面具']
[]
['防范']
[]
['防风']
[]
['防风林']
[]
['防腐']
[]
['防腐剂']
[]
['防寒']
[]
['防洪']
[]
['防护']
[]
['防护林']
[]
['防化兵']
[]
['防患未然']
[]
['防火墙']
[]
['防空']
[]
['防空洞']
[]
['防空壕']
[]
['防老']
[]
['防凌']
[]
['防区']
[]
['防身']
[]
['防守']
[]
['防暑']
[]
['防特']
[]
['防微杜渐']
[]
['防伪']
[]
['防卫']
[]
['防卫过当']
[]
['防务']
[]
['防线']
[]
['防汛']
[]
['防疫']
[]
['防御']
[]
['防震']
[]
['防止']
[]
['防治']
[]
['坊']
[]
[]
['妨']
[]
['妨碍']
[]
['妨害']
[]
['肪']
[]
['房']
[]
['房']
[]
['房本']
[]
['房补']
[]
['房舱']
[]
['房产']
[]
['房产主']
[]
['房颤']
[]
['房车']
[]
['房东']
[]
['房改']
[]
['房管']
[]
['房基']
[]
['房间']
[]
['房客']
[]
['房契']
[]
['房钱']
[]
['房山']
[]
['房市']
[]
['房事']
[]
['房贴']
[]
['房柁']
[]
['房屋']
[]
['房型']
[]
['房檐']
[]
['房

['风尘']
[]
['风驰电掣']
[]
['风传']
[]
['风吹草动']
[]
['风挡']
[]
['风刀霜剑']
[]
['风灯']
[]
['风笛']
[]
['风电']
[]
['风电场']
[]
['风洞']
[]
['风斗']
[]
['风度']
[]
['风发']
[]
['风帆']
[]
['风范']
[]
['风风火火']
[]
['风干']
[]
['风格']
[]
['风骨']
[]
['风光']
[]
['风光']
[]
['风害']
[]
['风寒']
[]
['风耗']
[]
['风和日丽']
[]
['风戽']
[]
['风花雪月']
[]
['风华']
[]
['风化']
[]
['风化']
[]
['风火墙']
[]
['风级']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['风纪']
[]
['风纪扣']
[]
['风景']
[]
['风景画']
[]
['风景线']
[]
['风镜']
[]
['风卷残云']
[]
['风口']
[]
['风口浪尖']
[]
['风浪']
[]
['风雷']
[]
['风力']
[]
['风力发电']
[]
['风凉']
[]
['风凉话']
[]
['风铃']
[]
['风流']
[]
['风流云散']
[]
['风马牛不相及']
[]
['风帽']
[]
['风貌']
[]
['风门']
[]
['风靡']
[]
['风魔']
[]
['风磨']
[]
['风鸟']
[]
['风平浪静']
[]
['风起云涌']
[]
['风气']
[]
['风琴']
[]
['风清弊绝']
[]
['风情']
[]
['风趣']
[]
['风圈']
[]
['风骚']
[]
['风骚']
[]
['风色']
[]
['风沙']
[]
['风扇']
[]
['风尚']
[]
['风声']
[]
['风声鹤唳']
[]
['风湿']
[]
['风湿病']
[]
['风蚀']
[]
['风势']
[]
['风霜']
[]
['风水']
[]
['风俗']
[]
['风俗画']
[]
['风瘫']
[]
['风调雨顺']
[]
['风头']
[]
['风头']
[]
['风土']
[]
['风味']
[]
['风闻']
[]
['风物']
[]
['风